# Quick demo of the package

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
import os 
os.chdir('..')
from fairness.metrics import *
from fairness.approximatemultiwasserstein import MultiWasserStein
from vizualization.vizualization import viz_fairness_distrib, waterfall_plot, arrow_plot, arrow_plot_permutations, permutations_cols, calculate_perm_wst
